In [376]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## 1. Read and split dataset

In [377]:
df = pd.read_csv('Dataset2.csv', encoding='latin1')
df

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,30/11/2018,1003,19,4.2,34,2.6,1894,-10.3,0.0,0.0,0.0,Autumn,No Holiday,Yes
8756,30/11/2018,764,20,3.4,37,2.3,2000,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8757,30/11/2018,694,21,2.6,39,0.3,1968,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8758,30/11/2018,712,22,2.1,41,1.0,1859,-9.8,0.0,0.0,0.0,Autumn,No Holiday,Yes


In [378]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       8760 non-null   object 
 1   Rented Bike Count          8760 non-null   int64  
 2   Hour                       8760 non-null   int64  
 3   Temperature(°C)            8760 non-null   float64
 4   Humidity(%)                8760 non-null   int64  
 5   Wind speed (m/s)           8760 non-null   float64
 6   Visibility (10m)           8760 non-null   int64  
 7   Dew point temperature(°C)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)    8760 non-null   float64
 9   Rainfall(mm)               8760 non-null   float64
 10  Snowfall (cm)              8760 non-null   float64
 11  Seasons                    8760 non-null   object 
 12  Holiday                    8760 non-null   object 
 13  Functioning Day            8760 non-null   objec

There are some non numeric features, so I will fix that. Also, I will create some new feature so the model can learn better.

In [379]:
# Convert Date to datetime and extract Day of the Week
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Create 2 features about week days and weekend
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# Create a feature about the differences between temperature and dew point temperature
df['Temp_DewPointDiff'] = df['Temperature(°C)'] - df['Dew point temperature(°C)']

# Categorize Hour into different time of the day
def categorize_hour(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['HourCategory'] = df['Hour'].apply(categorize_hour)

# Create interaction terms between temp and humid; temp and wind speed
df['Temp_Humidity'] = df['Temperature(°C)'] * df['Humidity(%)']
df['Temp_WindSpeed'] = df['Temperature(°C)'] * df['Wind speed (m/s)']


df.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,DayOfWeek,IsWeekend,Temp_DewPointDiff,HourCategory,Temp_Humidity,Temp_WindSpeed
0,2017-12-01,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,4,0,12.4,Night,-192.4,-11.44
1,2017-12-01,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,4,0,12.1,Night,-209.0,-4.40
2,2017-12-01,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,4,0,11.7,Night,-234.0,-6.00
3,2017-12-01,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,4,0,11.4,Night,-248.0,-5.58
4,2017-12-01,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,4,0,12.6,Night,-216.0,-13.80


In [380]:
# Encode the object dtype
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['Seasons'] = label_encoder.fit_transform(df['Seasons'])
df['Holiday'] = label_encoder.fit_transform(df['Holiday'])
df['HourCategory'] = label_encoder.fit_transform(df['HourCategory'])
df['Functioning Day'] = label_encoder.fit_transform(df['Functioning Day'])

display(df.head())

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,DayOfWeek,IsWeekend,Temp_DewPointDiff,HourCategory,Temp_Humidity,Temp_WindSpeed
0,2017-12-01,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,3,1,1,4,0,12.4,3,-192.4,-11.44
1,2017-12-01,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,3,1,1,4,0,12.1,3,-209.0,-4.40
2,2017-12-01,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,3,1,1,4,0,11.7,3,-234.0,-6.00
3,2017-12-01,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,3,1,1,4,0,11.4,3,-248.0,-5.58
4,2017-12-01,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,3,1,1,4,0,12.6,3,-216.0,-13.80


In [381]:
# Last 10 unique_dates 
unique_dates = df['Date'].unique()
unique_dates[-10:]

<DatetimeArray>
['2018-11-21 00:00:00', '2018-11-22 00:00:00', '2018-11-23 00:00:00',
 '2018-11-24 00:00:00', '2018-11-25 00:00:00', '2018-11-26 00:00:00',
 '2018-11-27 00:00:00', '2018-11-28 00:00:00', '2018-11-29 00:00:00',
 '2018-11-30 00:00:00']
Length: 10, dtype: datetime64[ns]

In [382]:
# Random split the data for train and test sets
train_set, test_set = train_test_split(df, test_size=0.3, random_state=42)

# Leave the last 10 unique_dates
train_set_2 = train_set[~train_set['Date'].isin(unique_dates[-10:])]
test_set_2 = test_set[~test_set['Date'].isin(unique_dates[-10:])]

display(train_set_2.shape)
display(test_set_2.shape)

(5971, 20)

(2549, 20)

## 2. Distribution for train and test set with respect to `Functioning Day`

In [383]:
display(train_set_2['Functioning Day'].value_counts())
display(train_set_2['Functioning Day'].value_counts(normalize=True))

Functioning Day
1    5761
0     210
Name: count, dtype: int64

Functioning Day
1    0.96483
0    0.03517
Name: proportion, dtype: float64

In [384]:
train_set_2[train_set_2['Functioning Day'] == 1].head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,DayOfWeek,IsWeekend,Temp_DewPointDiff,HourCategory,Temp_Humidity,Temp_WindSpeed
1444,2018-01-30,33,4,-11.1,50,1.2,1986,-19.4,0.00,0.0,0.0,3,1,1,1,0,8.3,3,-555.0,-13.32
1652,2018-02-07,218,20,-5.8,44,2.1,1994,-16.1,0.00,0.0,0.0,3,1,1,2,0,10.3,1,-255.2,-12.18
1893,2018-02-17,133,21,-2.3,38,2.3,2000,-14.7,0.00,0.0,0.0,3,0,1,5,1,12.4,3,-87.4,-5.29
3880,2018-05-11,1496,16,19.1,54,3.2,542,9.5,0.94,0.0,0.0,1,1,1,4,0,9.6,0,1031.4,61.12
2406,2018-03-11,63,6,3.4,85,1.5,234,1.1,0.00,0.0,0.0,1,1,1,6,1,2.3,2,289.0,5.10


In [385]:
train_set_2[train_set_2['Functioning Day'] == 0].head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,DayOfWeek,IsWeekend,Temp_DewPointDiff,HourCategory,Temp_Humidity,Temp_WindSpeed
7496,2018-10-09,0,8,11.4,66,0.8,1991,5.2,0.18,0.0,0.0,0,0,0,1,0,6.2,2,752.4,9.12
7336,2018-10-02,0,16,21.7,36,3.2,1929,5.9,1.64,0.0,0.0,0,1,0,1,0,15.8,0,781.2,69.44
7383,2018-10-04,0,15,25.3,39,1.6,2000,10.3,1.88,0.0,0.0,0,1,0,3,0,15.0,0,986.7,40.48
7244,2018-09-28,0,20,18.0,67,0.7,2000,11.7,0.00,0.0,0.0,0,1,0,4,0,6.3,1,1206.0,12.60
8251,2018-11-09,0,19,11.9,71,2.7,589,6.7,0.00,0.0,0.0,0,1,0,4,0,5.2,1,844.9,32.13


In [386]:
display(test_set_2['Functioning Day'].value_counts())
display(test_set_2['Functioning Day'].value_counts(normalize=True))

Functioning Day
1    2464
0      85
Name: count, dtype: int64

Functioning Day
1    0.966654
0    0.033346
Name: proportion, dtype: float64

In [387]:
test_set_2[test_set_2['Functioning Day'] == 0].head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,DayOfWeek,IsWeekend,Temp_DewPointDiff,HourCategory,Temp_Humidity,Temp_WindSpeed
7238,2018-09-28,0,14,19.4,53,1.4,2000,9.5,0.68,0.0,0.0,0,1,0,4,0,9.9,0,1028.2,27.16
8167,2018-11-06,0,7,8.3,84,0.9,422,5.7,0.00,0.0,0.0,0,1,0,1,0,2.6,2,697.2,7.47
7374,2018-10-04,0,6,13.2,73,1.1,1900,8.4,0.00,0.0,0.0,0,1,0,3,0,4.8,2,963.6,14.52
7018,2018-09-19,0,10,22.9,59,1.1,1819,14.4,1.49,0.0,0.0,0,1,0,2,0,8.5,2,1351.1,25.19
8252,2018-11-09,0,20,11.9,72,2.5,526,7.0,0.00,0.0,0.0,0,1,0,4,0,4.9,1,856.8,29.75


From what I understand, `Function Day` means that the days where the bike rental business is open. `Function Day` "No" means that day the service is not functioning. The `Rented Bike Count` on closed days is are all 0, because the rental service is not opened in that particular day. This is the same for the 2 test and train dataset. About the distribution, we can see that both sets have around 97% of sample data said "Yes" on `Function Day`, meaning the majority of times the bike rental business is in service.

## 3. Linear regression to predict `Wind Speed` 

In [388]:
X_train, y_train = train_set_2.drop('Wind speed (m/s)', axis=1), train_set_2['Wind speed (m/s)']
X_test, y_test = test_set_2.drop('Wind speed (m/s)', axis=1), test_set_2['Wind speed (m/s)']

X_train.drop('Date', axis=1, inplace=True)
X_test.drop('Date', axis=1, inplace=True)

In [389]:
# Features scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [390]:
# Fit the linear regression model
lr1 = LinearRegression()
lr1.fit(X_train_scaled, y_train)

# Get the coefficients and intercept
coefficients = lr1.coef_
intercept = lr1.intercept_
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [-0.0085859   0.11022794 -0.93266494 -0.11753696  0.05350959 -0.80371221
  0.12510473 -0.00617111  0.04338528  0.01979922 -0.02833547 -0.00903589
 -0.0700985   0.05518981 -0.13390471 -0.07479023  0.8165062   1.09051269]
Intercept: 1.7399765533411486


In [391]:
# Make predictions on the test set
y_pred = lr1.predict(X_test_scaled)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r_squared = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r_squared)

Mean Absolute Error (MAE): 0.4578128913674492
Mean Squared Error (MSE): 0.47931804765785047
Root Mean Squared Error (RMSE): 0.6923279913869224
R-squared (R²): 0.5661514042225981


So I have choose these 4 metrics to measure the performance of the linear regression model. Here are the explanation.

#### Mean Absolute Error (MAE)
A popular and easy interpret metric. This metric gives the absolute difference between predicted vs actual value, and since its an absolute difference. This metric is robust to outliers. A MAE of 0.72 indicates on average the predicted wind speed differs 0.458 m/s from actual wind speed.

#### Mean Squared Error (MSE) and Root Mean Squared Error (RMSE).
Mean Squared error penalizes the error by squaring the mean between predicted and actual values. Larger errors will be more penalized, and the goal is to lower the MSE as low as possible. RMSE derived from MSE, where it is the square root of MSE. It brings the error back to the original unit, it is easier to interpret than MSE, but still penalize large error. With MSE (0.48) and RMSE (0.69), in the context of wind speed, this error is large, but not too large. 

#### R-squared ($R^2$)
Measurement of the percentage of target variation that is explained by the model. In other words, it is the square correlation between the target values and the predicted target values. The range is from 0 to 1, the higher the value, the better. A $R^2$ of 0.566 shows the only 56.6% of the variance in wind speed is explained by the model. This indicates that we need more complex model since the data might not be linear.

## 4. Apply PCA and retrain the linear model

In [392]:
# Apply PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

pca.score(X_test_scaled)

-23.52756651572079

In [393]:
# Visualize PCA
import plotly.express as px
fig = px.scatter_3d(x=X_train_pca[:, 0], y=X_train_pca[:, 1], z=X_train_pca[:, 2], color=y_train)
fig.update_layout(
    title="PCA visualization",
    scene=dict(
        xaxis_title="First Principal Component",
        yaxis_title="Second Principal Component",
        zaxis_title="Third Principal Component"
    )
)
fig.show()

In [394]:
# Fit the linear regression model
lr2 = LinearRegression()
lr2.fit(X_train_pca, y_train)

# Get the coefficients and intercept
coefficients = lr2.coef_
intercept = lr2.intercept_
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [-0.02045406 -0.23821127  0.03417025]
Intercept: 1.7399765533411489


In [395]:
# Make predictions on the test set
y_pred = lr2.predict(X_test_pca)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r_squared = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r_squared)

Mean Absolute Error (MAE): 0.7311048744210239
Mean Squared Error (MSE): 0.8700081415565893
Root Mean Squared Error (RMSE): 0.9327422696311073
R-squared (R²): 0.21252326639144592


The performance with PCA is worst compare to the previous model. Let's take a closer look on while that is the case using explained variance.

In [397]:
explained_variance_ratio = pca.explained_variance_ratio_
print("Explained Variance Ratio:", explained_variance_ratio)

Explained Variance Ratio: [0.23378724 0.19903821 0.10051349]


We can see that we need more than 3 components to capture most of the variance. But using PCA is not going cut it (its just reducing the dimension), we need a more complex model.

## 5. Apply L2 regulariser
The question seems to be asking to apply L2 regulariser	on the logistic	regression model, but I think it might be a typo because this is a regression problem, not a classification. For that, I will be using L2 linear regression. L2 puts restriction on feature weights from taking on very large values.

The model seems to have high bias or under-fitting, so I will try to decrease the regularization parameter.

In [ ]:
from sklearn.linear_model import RidgeCV

lasso_cv = RidgeCV(alphas=np.logspace(-6, 1, 100), cv=5)
lasso_cv.fit(X_train_scaled, y_train)
best_alpha = lasso_cv.alpha_
print(f"Best alpha: {best_alpha}")

Best alpha: 1.4174741629268048


In [ ]:
from sklearn.linear_model import Ridge

ridgelr = Ridge(alpha=1.4174741629268048) # alpha in the function represents lambda

# Fit the data
ridgelr.fit(X_train_scaled, y_train)

# Do a prediction
y_pred = ridgelr.predict(X_test_scaled)

# Get the coefficients and intercept
coefficients = ridgelr.coef_
intercept = ridgelr.intercept_
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [-0.00906131  0.1103911  -0.92776243 -0.1157534   0.05379626 -0.79972053
  0.12448946 -0.00600458  0.04301859  0.020336   -0.02824271 -0.00890777
 -0.06979993  0.05472261 -0.13263094 -0.07514846  0.80924     1.08845495]
Intercept: 1.7399765533411486


In [ ]:
# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r_squared = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r_squared)

Mean Absolute Error (MAE): 0.4577858123419435
Mean Squared Error (MSE): 0.47938353371375697
Root Mean Squared Error (RMSE): 0.692375283869779
R-squared (R²): 0.566092130357287


The L2 regulariser performance on the metrics is not much different from regular linear regression model. Like I suspected, we need a model that can learn non-linear data to perform better. I will be using SVM. I will be using grid search to find the best parameter, and the fit the SVM model that the best hyperparameter.

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance

# Define the SVM model with RBF kernel
svr = SVR(kernel='rbf')

# Define the parameter grid for GridSearch
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, scoring='r2', cv=5)
grid_search.fit(X_train_scaled, y_train)

# Best parameter (C) and corresponding score
best_C = grid_search.best_params_['C']
best_gamma = grid_search.best_params_['gamma']

print(f"Best C: {best_C}")
print(f"Best Gamma: {best_gamma}")

Best C: 100
Best Gamma: 0.1


In [ ]:
# Train the best SVM model
best_svr = SVR(kernel='rbf', C=best_C, gamma=best_gamma)
svm_model = best_svr.fit(X_train_scaled, y_train)

# Get the support vectors, dual coefficients, and intercept
support_vectors = svm_model.support_vectors_
dual_coef = svm_model.dual_coef_
intercept = svm_model.intercept_

print("Support Vectors:", support_vectors)
print("Dual Coefficients:", dual_coef)
print("Intercept:", intercept)

Support Vectors: [[-0.75259228  1.23259374 -1.59305502 ... -0.64777597 -1.42675921
  -1.22746421]
 [-1.08668619 -0.51226572 -0.15468021 ...  0.23630868 -0.07583697
  -0.47484254]
 [-0.88285825  1.37799869 -1.30036247 ...  1.12039333 -1.20178427
  -0.98401054]
 ...
 [-0.92117178 -1.53010041 -1.28363718 ...  1.12039333 -1.2422744
  -0.85645353]
 [-0.01544014 -0.22145581  1.51784864 ...  0.23630868  1.06244507
   0.42335665]
 [-0.59320803  1.23259374 -1.39235156 ... -0.64777597 -1.31708728
  -0.92924229]]
Dual Coefficients: [[100.          -2.23058374  -6.59362574 ... -22.42100242  -0.20939939
  -17.79787247]]
Intercept: [2.01370951]


In [ ]:
# Calculate metrics
y_pred = best_svr.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r_squared = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r_squared)

Mean Absolute Error (MAE): 0.18070339833690724
Mean Squared Error (MSE): 0.1167137026118896
Root Mean Squared Error (RMSE): 0.34163387216710467
R-squared (R²): 0.8943580859648854


We can see that with SVM model, it can captures up to 89.44% target variation. The other metrics are also performing way better than linear models. That confirms my suspection that our data is non-linear. 